In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import numpy as np

def load_existing_results(file_path="forecasting_results_deep.json"):
    """
    Load existing results from a JSON file.
    Returns an empty dictionary if the file doesn't exist.
    """
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return json.load(f)
    return {}


def save_results_to_json(data, file_path="secured_forecasting_results.json"):
    """
    Save the results dictionary to a JSON file, handling NumPy data types.
    """

    # Handle NumPy data types (recursive conversion)
    def convert_numpy(obj):
        if isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(i) for i in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert arrays to lists
        else:
            return obj

    # Convert data and save to JSON
    data = convert_numpy(data)
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"✅ Results saved to {file_path}")



def store_results(dataset_name, horizons, horizon_value, experiment_type, backbone, mae_result, file_path="forecasting_results.json"):
    """
    Store MAE results for a given experiment type (stl_mae, mtl_mae, global_mae) per horizon.

    Args:
    - dataset_name (str): Name of the dataset (e.g., 'Solar', 'Air Quality').
    - horizons (list): List of horizon values (e.g., [1, 2, 4, 8, 16]).
    - horizon_value (int): The horizon corresponding to the mae_result provided.
    - experiment_type (str): One of ['stl_mae', 'mtl_mae', 'global_mae'].
    - backbone (str): Model backbone name (e.g., 'Deep_LSTM', 'simple_transformer').
    - mae_result (list): MAE values for the current horizon (list of floats).
    - file_path (str): JSON file to store the results.

    Returns:
    - None
    """
    # Load existing results
    results_dict = load_existing_results(file_path)

    # Create dataset entry if it doesn't exist
    dataset_key = f"{dataset_name}_{backbone}"
    if dataset_key not in results_dict:
        results_dict[dataset_key] = {
            "horizons": horizons,
            "mtl": [[] for _ in horizons],
            "global": [[] for _ in horizons],
            "independent": [[] for _ in horizons]
        }

    # Find index for the given horizon
    try:
        horizon_index = horizons.index(horizon_value)
    except ValueError:
        raise ValueError(f"⚠️ Horizon value {horizon_value} not found in {horizons}.")

    # Append the mae_result to the correct horizon
    results_dict[dataset_key][experiment_type][horizon_index].extend(mae_result)

    # Save updated results
    save_results_to_json(results_dict, file_path)

# Base Architecture

In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import dateutil
from sklearn.preprocessing import MinMaxScaler

# ------------------ MODEL COMPONENTS ------------------

class TaskSpecificAttention(nn.Module):
    def __init__(self, input_dim):
        super(TaskSpecificAttention, self).__init__()
        self.fc = nn.Linear(input_dim, input_dim)
        self.residual_fc = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        attention_weights = F.softmax(self.fc(x), dim=-1)
        context_vector = torch.tanh(x * attention_weights)
        return x + self.residual_fc(context_vector)


class SharedGlobalTemporalAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SharedGlobalTemporalAttention, self).__init__()
        self.fc = nn.Linear(hidden_dim, hidden_dim)
        self.final_fc = nn.Linear(hidden_dim, 1)
        self.residual_fc = nn.Linear(hidden_dim, hidden_dim)
        # Projection layer to match hidden_dim after concatenation
        self.projection_layer = nn.Linear(hidden_dim * 3, hidden_dim)

    def forward(self, x_list):
        combined_hidden = torch.stack([x.mean(dim=1) for x in x_list], dim=1).mean(dim=1)
        tanh_hidden = torch.tanh(self.fc(combined_hidden))
        attention_scores = self.final_fc(tanh_hidden).squeeze(-1)
        attention_weights = F.softmax(attention_scores, dim=-1).unsqueeze(-1)
        context_vector = combined_hidden * attention_weights
        repeated_context = self.residual_fc(context_vector).unsqueeze(1)
        # Concatenation and projection to hidden_dim
        enriched_context_list = [
            self.projection_layer(torch.cat((x, repeated_context.repeat(1, x.size(1), 1), x * repeated_context), dim=-1))
            for x in x_list
        ]
        return enriched_context_list


class FATHOMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, window_size=32):
        super(FATHOMModel, self).__init__()
        self.task_attention = TaskSpecificAttention(input_dim)
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim * 3, hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, shared_context):
        x = self.task_attention(x)
        x, _ = self.lstm1(x)
        enriched_context = torch.cat((x, shared_context, x * shared_context), dim=-1)
        x, _ = self.lstm2(enriched_context)
        x = torch.cat((x[:, -1, :], shared_context[:, -1, :]), dim=-1)
        return self.fc2(F.relu(self.fc1(x)))


class MultiTaskFATHOM(nn.Module):
    def __init__(self, num_tasks, input_dim, hidden_dim, output_dim, window_size=32):
        super(MultiTaskFATHOM, self).__init__()
        self.shared_global_attention = SharedGlobalTemporalAttention(hidden_dim)
        self.tasks = nn.ModuleList([
            FATHOMModel(input_dim, hidden_dim, output_dim, window_size) for _ in range(num_tasks)
        ])

    def forward(self, inputs):
        # n sites, n tensors: [batch_size, window_size, input_dim]
        # print(len(inputs))
        assert len(inputs) == len(self.tasks), (
            f"Mismatch: Received {len(inputs)} inputs but expected {len(self.tasks)} tasks."
        )
        first_stage_outputs = []
        for task_model, x in zip(self.tasks, inputs):
            x, _ = task_model.lstm1(task_model.task_attention(x))
            first_stage_outputs.append(x)

        shared_contexts = self.shared_global_attention(first_stage_outputs)
        outputs = []
        for i, (task_model, x, shared_context) in enumerate(zip(self.tasks, inputs, shared_contexts)):
            preds = task_model(x, shared_context)
            outputs.append(preds)
            # print(f"Task {i + 1}: Output shape {preds.shape}")
        return outputs


# ------------------ DATA LOADER ------------------

def df_to_X_y(df, features, target, window_size=32, horizon=1):
    if target not in features:
        features = [target] + features

    data = df[features].to_numpy()
    target_data = df[target].to_numpy()

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i + window_size])
        y.append(target_data[i + window_size: i + window_size + horizon])

    return np.array(X), np.array(y)


def load_and_preprocess_site_data(site_path, features, target, window_size=32, horizon=1, min_date=None, max_date=None, batch_size=16, device='cpu'):
    df = pd.read_csv(site_path)

    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        if min_date:
            min_date = dateutil.parser.parse(min_date) if isinstance(min_date, str) else min_date
            df = df[df['date'] >= min_date]
        if max_date:
            max_date = dateutil.parser.parse(max_date) if isinstance(max_date, str) else max_date
            df = df[df['date'] <= max_date]
        df.drop(columns=['date'], inplace=True)

    if target not in features:
        features = [target] + features

    all_columns = features
    if not all(col in df.columns for col in all_columns):
        missing = [col for col in all_columns if col not in df.columns]
        raise ValueError(f"Missing columns in dataset: {missing}")

    train_size = int(0.8 * len(df))
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]

    val_size = int(0.2 * len(train_df))
    train_df, val_df = train_df.iloc[:-val_size], train_df.iloc[-val_size:]

    print(f"Train size: {len(train_df)} | Validation size: {len(val_df)} | Test size: {len(test_df)}")

    train_mean, train_std = train_df[all_columns].mean(), train_df[all_columns].std()
    train_df[all_columns] = (train_df[all_columns] - train_mean) / (train_std + 1e-8)
    val_df[all_columns] = (val_df[all_columns] - train_mean) / (train_std + 1e-8)
    test_df[all_columns] = (test_df[all_columns] - train_mean) / (train_std + 1e-8)
    
    # scaler = MinMaxScaler()
    # train_df[all_columns] = scaler.fit_transform(train_df[all_columns])
    # val_df[all_columns] = scaler.transform(val_df[all_columns])
    # test_df[all_columns] = scaler.transform(test_df[all_columns])

    X_train, y_train = df_to_X_y(train_df, features, target, window_size, horizon)
    X_val, y_val = df_to_X_y(val_df, features, target, window_size, horizon)
    X_test, y_test = df_to_X_y(test_df, features, target, window_size, horizon)

    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device))
    val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32).to(device), torch.tensor(y_val, dtype=torch.float32).to(device))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.float32).to(device))

    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, drop_last=True)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)

    return train_loader, val_loader, test_loader

# ------------------ TRAINING & EVALUATION ------------------

def train_fathom_model(site_loaders, input_dim, hidden_dim, output_dim, num_epochs, window_size ,device):
    
    num_tasks = len(site_loaders)
    
    model = MultiTaskFATHOM(num_tasks, input_dim, hidden_dim, output_dim, window_size).to("cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
    criterion = nn.MSELoss()
    
    # Unpack loaders for each task
    train_loaders = [loader_tuple[0] for loader_tuple in site_loaders]
    val_loaders = [loader_tuple[1] for loader_tuple in site_loaders]
    
    for epoch in range(num_epochs):
        model.train()
        train_losses = []
        # Iterate over batches from all tasks simultaneously
        for batches in zip(*train_loaders):
            # Each batch in batches is a tuple (X, y) for a given task
            Xs = [batch[0].to(device) for batch in batches]
            ys = [batch[1].to(device) for batch in batches]
            
            optimizer.zero_grad()
            # Pass the list of task batches to the model
            preds_list = model(Xs)  # expects a list of tensors, one per task
            
            # Compute losses for each task and sum them
            losses = [
                criterion(pred, y.view(y.size(0), -1))
                for pred, y in zip(preds_list, ys)
            ]
            total_loss = sum(losses)
            total_loss.backward()
            optimizer.step()
            train_losses.append(total_loss.item())
        
        # Validation phase (similarly, iterate over all task validation loaders)
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batches in zip(*val_loaders):
                Xs = [batch[0].to(device) for batch in batches]
                ys = [batch[1].to(device) for batch in batches]
                preds_list = model(Xs)
                losses = [
                    criterion(pred, y.view(y.size(0), -1)).item()
                    for pred, y in zip(preds_list, ys)
                ]
                # Average loss over tasks for this batch
                val_losses.append(sum(losses) / num_tasks)
        
        scheduler.step()
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {np.mean(train_losses):.4f} | Validation Loss: {np.mean(val_losses):.4f}")
    
    print("Training complete.")
    return model


def evaluate_fathom_model(model, site_loaders, device='cpu'):
    model.eval()
    # Prepare test loaders from site_loaders
    test_loaders = [loader_tuple[2] for loader_tuple in site_loaders]
    task_preds, task_targets = [[] for _ in range(len(test_loaders))], [[] for _ in range(len(test_loaders))]
    
    with torch.no_grad():
        for batches in zip(*test_loaders):
            Xs = [batch[0].to(device) for batch in batches]
            ys = [batch[1].to(device) for batch in batches]
            preds_list = model(Xs)
            for i, (pred, y) in enumerate(zip(preds_list, ys)):
                task_preds[i].append(pred.cpu().numpy())
                task_targets[i].append(y.cpu().numpy())
    
    # Compute MAE for each task
    mae_scores = []
    for preds, targets in zip(task_preds, task_targets):
        preds_concat = np.concatenate(preds)
        targets_concat = np.concatenate(targets)
        mae_scores.append(mean_absolute_error(targets_concat, preds_concat))
    
    print("Evaluation complete.")
    return mae_scores

import numpy as np
import matplotlib.pyplot as plt

def plot_predictions_vs_ground_truth(model, site_loaders, device='cpu', window_size=32, stride=16):
    """
    Plots the model predictions vs. ground truth for each task using the test DataLoader.
    
    Args:
        model: Trained multi-task model.
        site_loaders: List of tuples (train_loader, val_loader, test_loader) for each site.
        device: "cpu" or "cuda".
        window_size: Sliding window size used during training.
        stride: Plot every 'stride'-th point to reduce clutter.
    """
    model.eval()
    # Extract test loaders from each (train, val, test) tuple
    test_loaders = [loaders[2] for loaders in site_loaders]

    # Collect predictions and ground truth from each task
    all_preds = [[] for _ in range(len(test_loaders))]
    all_truth = [[] for _ in range(len(test_loaders))]

    with torch.no_grad():
        # Zip all test loaders to get one batch per task at each iteration
        for batches in zip(*test_loaders):
            # For each task, extract inputs and targets
            Xs = [batch[0].to(device) for batch in batches]
            ys = [batch[1].to(device) for batch in batches]

            # Forward pass for all tasks
            preds_list = model(Xs)

            # Accumulate predictions and ground truths
            for i in range(len(test_loaders)):
                all_preds[i].append(preds_list[i].cpu().numpy())
                all_truth[i].append(ys[i].cpu().numpy())

    # Now plot for each task
    for i in range(len(test_loaders)):
        # Concatenate along batch dimension and flatten
        preds_i = np.concatenate(all_preds[i], axis=0).flatten()
        truth_i = np.concatenate(all_truth[i], axis=0).flatten()

        # Truncate to the same length, just in case
        min_len = min(len(preds_i), len(truth_i))
        preds_i = preds_i[:min_len]
        truth_i = truth_i[:min_len]

        # Build x-axes
        # Ground truth covers indices [0, 1, 2, ..., min_len-1]
        # Predictions are typically "window_size" steps ahead
        time_axis = np.arange(min_len)
        pred_axis = time_axis + window_size  # shift by window_size

        # Downsample for plotting clarity (optional)
        time_axis_plot = time_axis[::stride]
        truth_plot = truth_i[::stride]
        pred_axis_plot = pred_axis[::stride]
        preds_plot = preds_i[::stride]

        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(time_axis_plot, truth_plot, label="Ground Truth", linewidth=1)
        plt.plot(pred_axis_plot, preds_plot, label="Predictions", linewidth=1, linestyle='--')
        plt.title(f"Task {i+1} Predictions vs Ground Truth")
        plt.xlabel("Sample Index")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.show()


# Usage

In [5]:
# if __name__ == "__main__":
#     # base_path = '../processed_ds/air_quality_cluster'
#     # features = ['PM2.5', 'OT', 'PM10', 'NO2']
#     # target = 'PM2.5'
#     # min_date = "2014-09-01"
#     # max_date =  "2014-11-12 19:00"
    
#     # ds = 'Solar_feb_22'
#     # features =  ['loc-1', 'loc-2', 'loc-3', 'loc-4']
#     # target = 'loc-1'
#     # base_path =  "../processed_ds/solar/"
#     # min_date = "2006-09-01"
#     # max_date= "2006-09-08 4:50"
    
#     ds = 'Crypto_feb_22'
#     features =  ['Open', 'High', 'Low', 'OT', 'Volume']
#     target = 'OT'
#     base_path =  "../processed_ds/crypto-data/"
#     min_date = "2018-04-01"
#     max_date= "2018-06-15"
    
#     site_paths = [
#         os.path.join(root, file)
#         for root, dirs, files in os.walk(base_path)
#         if root != base_path  # Exclude files in the base directory
#         for file in files
#         if file.endswith(".csv")
#     ]

#     total_sites = len(site_paths)
    
#     num_tasks = total_sites
#     batch_size, window_size, input_dim, hidden_dim, output_dim = 32, 32, len(features), 64, 16

#     # model = MultiTaskFATHOM(num_tasks, input_dim, hidden_dim, output_dim, window_size).to("cpu")
#     # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
#     # criterion = nn.MSELoss()

#     site_loaders = [load_and_preprocess_site_data(site_path, 
#                                                   features, 
#                                                   target, 
#                                                   window_size, 
#                                                   horizon=output_dim, 
#                                                   batch_size=batch_size, 
#                                                   min_date=min_date,
#                                                   max_date=max_date) for site_path in site_paths]

#     model = train_fathom_model(site_loaders, input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_epochs=5, window_size=window_size, device='')
#     mae_scores = evaluate_fathom_model(model, site_loaders, device="cpu")
#     print(f"MAE per task: {mae_scores}")
#     # Plot the predictions vs ground truth for each task
#     # plot_predictions_vs_ground_truth(model, site_loaders, device="cpu")

# Single Task Baseline

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import dateutil
from sklearn.preprocessing import MinMaxScaler

# ------------------ BASELINE SINGLE-TASK MODEL ------------------

class SingleTaskModel(nn.Module):
    """
    Baseline model with NO information sharing between tasks. Each task runs independently.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, window_size=32):
        super(SingleTaskModel, self).__init__()
        self.task_attention = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Softmax(dim=-1)
        )
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        attention_weights = self.task_attention(x)
        x = x * attention_weights  # Element-wise feature weighting
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]  # Last time step output
        x = F.relu(self.fc1(x))
        return self.fc2(x)


# ------------------ TRAINING & EVALUATION FOR BASELINE ------------------

def train_STL_model(site_loaders, input_dim, hidden_dim, output_dim, num_epochs=5, device='cpu'):
    """
    Trains independent models per site (NO information sharing between tasks).
    """
    num_tasks = len(site_loaders)
    models = [SingleTaskModel(input_dim, hidden_dim, output_dim).to(device) for _ in range(num_tasks)]
    optimizers = [torch.optim.Adam(model.parameters(), lr=0.001) for model in models]
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        for task_id, (train_loader, val_loader, _) in enumerate(site_loaders):
            models[task_id].train()
            train_losses = []

            for X, y in train_loader:
                X, y = X.to(device), y.to(device)
                optimizers[task_id].zero_grad()
                preds = models[task_id](X)
                loss = criterion(preds, y.view(y.size(0), -1))
                loss.backward()
                optimizers[task_id].step()
                train_losses.append(loss.item())

            print(f"Epoch {epoch + 1}/{num_epochs} | Task {task_id + 1} | Train Loss: {np.mean(train_losses):.4f}")

    print("Baseline training complete.")
    return models


def evaluate_stl_model(models, site_loaders, device='cpu'):
    """
    Evaluates independent models per site and computes MAE.
    """
    mae_scores = []

    for task_id, (_, _, test_loader) in enumerate(site_loaders):
        models[task_id].eval()
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                preds = models[task_id](X)
                all_preds.append(preds.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_concat = np.concatenate(all_preds, axis=0)
        targets_concat = np.concatenate(all_targets, axis=0)
        mae = mean_absolute_error(targets_concat, preds_concat)
        mae_scores.append(mae)
        print(f"Task {task_id + 1} - Baseline MAE: {mae:.4f}")

    print("Baseline evaluation complete.")
    return mae_scores

# Experiment

In [7]:
import os
datasets = [
    {
        'ds': 'Solar',
        'features': ['loc-1', 'loc-2', 'loc-3', 'loc-4'],
        'target': 'loc-1',
        'base_path': "../processed_ds/solar/",
        'min_date': "2006-09-01",
        'max_date': "2006-09-08 4:50"
    },
    {
        'ds': 'Air Quality',
        'features': ['PM2.5', 'OT', 'PM10', 'NO2'],
        'target': 'PM2.5',
        'base_path': '../processed_ds/air_quality_cluster',
        'min_date': "2014-09-01",
        'max_date': "2014-11-12 19:00"
    },
    {
        'ds': 'Crypto',
        'features': ['Open', 'High', 'Low', 'OT', 'Volume'],
        'target': 'OT',
        'base_path': "../processed_ds/crypto-data/",
        'min_date': "2018-04-01",
        'max_date': "2018-06-15"
    },
    # {
    #     'ds': 'Sales',
    #     'features': ['OT', 'customers', 'open', 'promo', 'holiday'],
    #     'target': 'OT',
    #     'base_path': "../processed_ds/stores_data/",
    #     'min_date': "2013-01-16",
    #     'max_date': "2015-07-31"
    # }
]

horizons = [1, 2, 4, 8, 16]

for dataset in datasets:
    print(f"\n==================== 🌟 DATASET: {dataset['ds']} ====================")
    for horizon in horizons:
        print(f"\n==================== ⏳ HORIZON: {horizon} ====================")
        
        site_paths = [
            os.path.join(root, file)
            for root, dirs, files in os.walk(dataset['base_path'])
            if root != dataset['base_path']
            for file in files
            if file.endswith(".csv")
        ]

        total_sites = len(site_paths)
        num_tasks = total_sites
        batch_size, window_size, input_dim, hidden_dim, output_dim = 32, 32, len(dataset['features']), 64, horizon

        site_loaders = [
            load_and_preprocess_site_data(
                site_path,
                dataset['features'],
                dataset['target'],
                window_size,
                horizon=output_dim,
                batch_size=batch_size,
                min_date=dataset['min_date'],
                max_date=dataset['max_date']
            ) for site_path in site_paths
        ]

        # Train and evaluate baseline models for comparison
        baseline_stl_models = train_STL_model(site_loaders, input_dim, hidden_dim, output_dim, num_epochs=5, device='cpu')
        baseline_stl_mae = evaluate_stl_model(baseline_stl_models, site_loaders, device='cpu')

        print(f"✅ Completed STL : {dataset['ds']} | Horizon: {horizon} | Baseline MAE per task: {baseline_stl_mae}")
        store_results(dataset_name=dataset['ds'],
                      horizons=[1,2,4,8,16],
                      horizon_value=horizon,
                      experiment_type='independent',
                      mae_result=baseline_stl_mae,
                      backbone='deep_lstm',
                      file_path='deep_forecasting_results.json'
                    )
        # Train MTL
        mtl_model = train_fathom_model(site_loaders, input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_epochs=5, window_size=window_size, device='cpu')
        mtl_scores = evaluate_fathom_model(mtl_model, site_loaders, device="cpu")
        print(f"✅ Completed MTL : {dataset['ds']} | Horizon: {horizon} | MTL MAE per task: {mtl_scores}")
        store_results(dataset_name=dataset['ds'],
                      horizons=[1,2,4,8,16],
                      horizon_value=horizon,
                      experiment_type='mtl',
                      mae_result=mtl_scores,
                      backbone='deep_lstm',
                      file_path='deep_forecasting_results.json'
                    )
        
print("\n🏆 All experiments completed successfully!")


==================== 🌟 DATASET: Solar ====================

==================== ⏳ HORIZON: 1 ====================
Train size: 1328 | Validation size: 332 | Test size: 415
Train size: 1328 | Validation size: 332 | Test size: 415
Train size: 1328 | Validation size: 332 | Test size: 415
Train size: 1328 | Validation size: 332 | Test size: 415
Train size: 1328 | Validation size: 332 | Test size: 415
Train size: 1328 | Validation size: 332 | Test size: 415
Epoch 1/5 | Task 1 | Train Loss: 0.7238
Epoch 1/5 | Task 2 | Train Loss: 0.6652
Epoch 1/5 | Task 3 | Train Loss: 0.6702
Epoch 1/5 | Task 4 | Train Loss: 0.6366
Epoch 1/5 | Task 5 | Train Loss: 0.6782
Epoch 1/5 | Task 6 | Train Loss: 0.7821
Epoch 2/5 | Task 1 | Train Loss: 0.2324
Epoch 2/5 | Task 2 | Train Loss: 0.2634
Epoch 2/5 | Task 3 | Train Loss: 0.1676


KeyboardInterrupt: 